In [1]:
import secrets
import json
from pathlib import Path
import pprint
import pdb
from typing import Any

from etl import markdown, pdfs, shared, videos

import docstore
import vecstore
from utils import pretty_log

pp = pprint.PrettyPrinter(indent=2)

import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import pipeline, TextStreamer
import json
import textwrap
from langchain.llms.huggingface_pipeline import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, RetrievalQA
from langchain.memory import ConversationBufferMemory
import langchain
import time

from langchain.chains.conversational_retrieval.prompts import CONDENSE_QUESTION_PROMPT
from langchain.chains.qa_with_sources.loading import load_qa_with_sources_chain
from langchain.chains.conversational_retrieval.base import ConversationalRetrievalChain


%load_ext autoreload
from QuestionAndAnswerUtils import *

In [2]:
securityChatbot = QuestionAndAnsweringCustomLlama2(QuestionRewritingPrompt=QuestionAndAnsweringCustomLlama2.QUESTION_REWRITING_TYPE.QUESTION_REWRITING_DEFAULT,
                                 QuestionAnsweringPrompt=QuestionAndAnsweringCustomLlama2.SECURITY_PROMPT_TYPE.PROMPT_TYPE_SECURITY_OFFICER_WITH_RAG_MEMORY_NOSOURCES,
                                 ModelType=QuestionAndAnsweringCustomLlama2.LLAMA2_VERSION_TYPE.LLAMA2_7B_chat,
                                debug=True, streamingOnAnotherThread=True)



securityChatbot.test_vectorDatasets_similarityScores_and_responses_no_memory(run_llm_chain=False)

securityChatbot.ask_question_and_streamtoconsole("What models use human instructions?")
securityChatbot.ask_question_and_streamtoconsole("Which are the advantage of each of these models?")
securityChatbot.ask_question_and_streamtoconsole("What are the downsides of your last model suggested above ?")



Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

🥞: found 35 vectors to search over in the database 
🥞: selecting sources by similarity to query 
[(Document(page_content='helps users track information about the health of their mouth\nand use appropriate toothbrushing programs.\n(4)SmartKettle - An smart kettle Application that speeds up and\nautomates the preparation of drinks.\n(5)SmartTV - An smart TV application where users can have\nindividual profiles and, based on their behavior, a recommen-\ndation system is used to recommend movies or TV shows.\n1The repository containing the applications and the Hub can be accesed at this link\nhttps://github.com/unibuc-cs/IoT-application-set.', metadata={'source': None, 'page': 2, 'is_endmatter': False, 'title': 'notknown', 'full-title': 'notknown - p2', 'sha256': 'e4b1f26f14a9d5db3ced61a81848ad1956f47072d22fe596bff591d913d59028', 'ignore': False}), 1.6027973), (Document(page_content='data that enables better decision making. In both academic\nand industrial initiatives, a variety of device

' Standalone question: What are the downsides of the IoT application set repository suggested above?</s>'

[llm/end] [1:chain:ConversationalRetrievalChain > 2:chain:LLMChain > 3:llm:HuggingFacePipeline] s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "  Standalone question: What are the downsides of the IoT application set repository suggested above?",
        "generation_info": null,
        "type": "Generation"
      }
    ]
  ],
  "llm_output": null,
  "run": null
}
[chain/end] [1:chain:ConversationalRetrievalChain > 2:chain:LLMChain] s] Exiting Chain run with output:
{
  "text": "  Standalone question: What are the downsides of the IoT application set repository suggested above?"
}


In [3]:

securityChatbot.simulate_raq_question("can you show me a resource utilization graph comparison between a normal session and current situation?", run_llm_chain=True)


🥞: selecting sources by similarity to query 
[(Document(page_content='the temperature. When the hub receives this data, it stores it so\nthat it can be used by another rule. According to rule 2, if the\ntemperature provided by FlowerPower is above 30 degrees, then\nWindWow luminosity is reduced by half. The luminosity is also a\nvalue that is tracked by the hub. It also stores it and is immediately\nused by rule 3. When the luminosity is below 30, the plant lamp is\nturned on by FlowerPower .\n5 EVALUATION\nTo provide a basis for problem discovery, we have introduced in-\ntentional problems in the smart applications and automation rules.\nAt this point, it is important to note that the applications already\ncontained a significant number of real issues identified by our frame-\nwork. In what follows, we categorise the issues based on the level of\ninteraction at which they occur, rather than the technical or logical\ncause. This decision illustrates our focus on identifying problems\nt

In [7]:
securityChatbot.simulate_raq_question("What Smart IOT applications are developed by Rares, Alin, and Ciprian Paduraru?", run_llm_chain=True)

🥞: selecting sources by similarity to query 
[(Document(page_content='helps users track information about the health of their mouth\nand use appropriate toothbrushing programs.\n(4)SmartKettle - An smart kettle Application that speeds up and\nautomates the preparation of drinks.\n(5)SmartTV - An smart TV application where users can have\nindividual profiles and, based on their behavior, a recommen-\ndation system is used to recommend movies or TV shows.\n1The repository containing the applications and the Hub can be accesed at this link\nhttps://github.com/unibuc-cs/IoT-application-set.', metadata={'source': None, 'page': 2, 'is_endmatter': False, 'title': 'notknown', 'full-title': 'notknown - p2', 'sha256': 'e4b1f26f14a9d5db3ced61a81848ad1956f47072d22fe596bff591d913d59028', 'ignore': False}), 0.78705966), (Document(page_content='TABLE I\nAPPLICATION SETDEVELOPMENT YEARS\nApplication Set\nYears No. of projects Valid apps Selected apps\nFlowerPower, Smarteeth,\n2020-2021 54 10 SmartKett

In [ ]:
start=time.time()
securityChatbot.ask_question("Give me some indications to solve a denial of service attack.")
securityChatbot.ask_question("Give me a few tools for this please")
securityChatbot.ask_question("Which one works best on Windows machines?")


end=time.time()

print(f"Total time: {end-start}")